In [156]:
import csv
import requests
import re
import os
import time
import pickle
import pandas as pd

LOG = 'D:/Thesis_data/log.txt'
def parse_identity(file1):
    hand=open(file1, encoding='utf-8', errors="replace")
    IDENTITY={}
    for line in hand:
        line=line.strip()
        if re.findall('^COMPANY CONFORMED NAME:',line):
            k = line.find(':')
            comnam=line[k+1:]
            comnam=comnam.strip()
            IDENTITY['name']=str(comnam)                                       
            break
        
    hand.seek(0)
    for line in hand:
        line=line.strip()
        if re.findall('^CENTRAL INDEX KEY:',line):
            k = line.find(':')
            cik=line[k+1:]
            cik=cik.strip()
            IDENTITY['cik']=str(cik)
            break
        
    hand.seek(0)
    for line in hand:
        line=line.strip()
        if re.findall('^STANDARD INDUSTRIAL CLASSIFICATION:',line):
            k = line.find(':')
            sic=line[k+1:]
            sic=sic.strip()
            siccode=[]
            for s in sic: 
                if s.isdigit():
                    siccode.append(s)    
            IDENTITY['sic']=''.join(siccode)
            break
        
    hand.seek(0)
    for line in hand:
        line=line.strip()
        if re.findall('^CONFORMED SUBMISSION TYPE:',line):
            k = line.find(':')
            subtype=line[k+1:]
            subtype=subtype.strip()
            IDENTITY['subtype']=str(subtype)
            break
            
    hand.seek(0)
    for line in hand:
        line=line.strip()
        if re.findall('^CONFORMED PERIOD OF REPORT:',line):
            k = line.find(':')
            cper=line[k+1:]
            cper=cper.strip()
            IDENTITY['CONFORMED DATE']=str(cper)
            break
            
    hand.seek(0)
    for line in hand:
        line=line.strip()
        if re.findall('^FILED AS OF DATE:',line):
            k = line.find(':')
            fdate=line[k+1:]
            fdate=fdate.strip()
            #print fdate                                
            IDENTITY['FILE DATE']=str(fdate)
            break
    
    hand.seek(0)
    for line in hand:
        line=line.strip()
        if re.findall('^FISCAL YEAR END:',line):
            line=line.strip()
            yearend=line[-4:]
            IDENTITY['yearend']= str(yearend)
            break        
    
    hand.close()
    return IDENTITY

def headerclean(content):
    mark0=0
    strings1=['</SEC-HEADER>','</IMS-HEADER>']
    for x, line in enumerate(content.split('\n')):
        line=line.strip()
        if any(s in line for s in strings1):
            mark0=x
            break
    return '\n'.join(content.split('\n')[mark0:])
    
def xbrl_clean(cond1, cond2, str0):
    locations=[0]
    #print locations
    placement1=[]
    str0=str0.lower()
    for m in re.finditer(cond1, str0):
        a=m.start()
        placement1.append(a)
    #print placement1
    
    if placement1!=[]:
        placement2=[]
        for m in re.finditer(cond2, str0):
            a=m.end()
            placement2.append(a)
    #    print placement2
        
        len1=len(placement1)
        placement1.append(len(str0))
        
        for i in range(len1):
            placement3=[]
            locations.append(placement1[i])
            for j in placement2:
                if (j>placement1[i] and j<placement1[i+1]):
                    placement3.append(j)
                    break
            if placement3!=[]:
                locations.append(placement3[0])
            else:
                locations.append(placement1[i])
    
    #print locations
    return locations

def asciireplace(con1, con2, content):
        output=content
        locations_xbrlbig=xbrl_clean(con1, con2, output)
        locations_xbrlbig.append(len(output))
        if locations_xbrlbig!=[0]:
            content=""
            if len(locations_xbrlbig)%2==0:
                for i in range(0,len(locations_xbrlbig),2):
                    content=content+output[locations_xbrlbig[i]:locations_xbrlbig[i+1]]
        return content    

def asciiclean(content):
    
    content = asciireplace("<type>zip", "</document>", content)
    content = asciireplace("<type>graphic", "</document>", content)
    content = asciireplace("<type>excel", "</document>", content)
    content = asciireplace("<type>pdf", "</document>", content)
    content = asciireplace("<type>xml", "</document>", content)
    content = asciireplace("<type>ex", "</document>", content)  
    return content

def tagsclean(content):
    p = re.compile(r'(<DIV.*?>)|(<DIV\n.*?>)|(<DIV\n\r.*?>)|(<DIV\r\n.*?>)|(<DIV.*?\n.*?>)|(<DIV.*?\n\r.*?>)|(<DIV.*?\r\n.*?>)')
    content=p.sub("",content)
    p = re.compile(r'(<div.*?>)|(<div\n.*?>)|(<div\n\r.*?>)|(<div\r\n.*?>)|(<div.*?\n.*?>)|(<div.*?\n\r.*?>)|(<div.*?\r\n.*?>)')
    content=p.sub("",content)
    p = re.compile(r'(<TD.*?>)|(<TD\n.*?>)|(<TD\n\r.*?>)|(<TD\r\n.*?>)|(<TD.*?\n.*?>)|(<TD.*?\n\r.*?>)|(<TD.*?\r\n.*?>)')
    content=p.sub("",content)
    p = re.compile(r'(<td.*?>)|(<td\n.*?>)|(<td\n\r.*?>)|(<td\r\n.*?>)|(<td.*?\n.*?>)|(<td.*?\n\r.*?>)|(<td.*?\r\n.*?>)')
    content=p.sub("",content)
    p = re.compile(r'(<TR.*?>)|(<TR\n.*?>)|(<TR\n\r.*?>)|(<TR\r\n.*?>)|(<TR.*?\n.*?>)|(<TR.*?\n\r.*?>)|(<TR.*?\r\n.*?>)')
    content=p.sub("",content)
    p = re.compile(r'(<tr.*?>)|(<tr\n.*?>)|(<tr\n\r.*?>)|(<tr\r\n.*?>)|(<tr.*?\n.*?>)|(<tr.*?\n\r.*?>)|(<tr.*?\r\n.*?>)')
    content=p.sub("",content)
    p = re.compile(r'(<FONT.*?>)|(<FONT\n.*?>)|(<FONT\n\r.*?>)|(<FONT\r\n.*?>)|(<FONT.*?\n.*?>)|(<FONT.*?\n\r.*?>)|(<FONT.*?\r\n.*?>)')
    content=p.sub("",content)
    p = re.compile(r'(<font.*?>)|(<font\n.*?>)|(<font\n\r.*?>)|(<font\r\n.*?>)|(<font.*?\n.*?>)|(<font.*?\n\r.*?>)|(<font.*?\r\n.*?>)')
    content=p.sub("",content)
    p = re.compile(r'(<P.*?>)|(<P\n.*?>)|(<P\n\r.*?>)|(<P\r\n.*?>)|(<P.*?\n.*?>)|(<P.*?\n\r.*?>)|(<P.*?\r\n.*?>)')
    content=p.sub("",content)
    p = re.compile(r'(<p.*?>)|(<p\n.*?>)|(<p\n\r.*?>)|(<p\r\n.*?>)|(<p.*?\n.*?>)|(<p.*?\n\r.*?>)|(<p.*?\r\n.*?>)')
    content=p.sub("",content)
    content=content.replace("</DIV>","")
    content=content.replace("</div>","")
    content=content.replace("</TR>","")
    content=content.replace("</tr>","")
    content=content.replace("</TD>","")
    content=content.replace("</td>","")
    content=content.replace("</FONT>","")
    content=content.replace("</font>","")
    content=content.replace("</P>","")
    content=content.replace("</p>","")
    return content

def table_clean(cond1, cond2, str1):
    Items0=["item 7", "item7", "item8", "item 8"]
    Items1=["item 1", "item 2","item 3","item 4","item 5","item 6","item 9", "item 10", "item1", "item2","item3","item4","item5","item6","item9", "item10"]
    
    str2=str1.lower()
    placement1=[]
    for m in re.finditer(cond1, str2):
        a=m.start()
        placement1.append(a)
    n=len(placement1)
    placement1.append(len(str2))
    
    placement2=[]
    for m in re.finditer(cond2, str2):
        a=m.end()
        placement2.append(a)
        
    if (placement1!=[] and placement2!=[]):
        current=str1[0:placement1[0]]
        
        for i in range(n):
            begin=placement1[i]
            for j in placement2:
                if j>begin:
                    end=j
                    break
            
            if end=="":
                current=current+str1[begin:placement1[i+1]]
            else:
                str2=""
                str2=str1[begin:end].lower()
                str2=str2.replace("&nbsp;"," ")
                str2=str2.replace("&NBSP;"," ")
                p = re.compile(r'&#\d{1,5};')
                str2=p.sub("",str2)
                p = re.compile(r'&#.{1,5};')
                str2=p.sub("",str2)
                if any(s in str2 for s in Items0):
                    if not any(s in str2 for s in Items1):
                        current=current+str2
                    
                current=current+str1[end:placement1[i+1]]
                end=""
    else:
        current=str1
    return current

def cleannewline(content):
    content=content.replace("\r\n"," ")
    p = re.compile(r'<.*?>')
    content=p.sub("",content)
    return content

def cleanothers(content):
    content=content.replace("&nbsp;"," ")
    content=content.replace("&NBSP;"," ")
    content=content.replace("&LT;","LT")
    content=content.replace("&#60;","LT")
    content=content.replace("&#160;"," ")
    content=content.replace("&AMP;","&")
    content=content.replace("&amp;","&")
    content=content.replace("&#38;","&")
    content=content.replace("&APOS;","'")
    content=content.replace("&apos;","'")
    content=content.replace("&#39;","'")
    content=content.replace('&QUOT;','"')
    content=content.replace('&quot;','"')
    content=content.replace('&#34;','"')
    content=content.replace("\t"," ")
    content=content.replace("\v","")
    content=content.replace("&#149;"," ")
    content=content.replace("&#224;","")
    content=content.replace("&#145;","")
    content=content.replace("&#146;","")
    content=content.replace("&#147;","")
    content=content.replace("&#148;","")
    content=content.replace("&#151;"," ")
    content=content.replace("&#153;","") 
    content=content.replace("&#111;","")
    content=content.replace("&#153;","")
    content=content.replace("&#253;","")
    content=content.replace("&#8217;","")
    content=content.replace("&#32;"," ")
    content=content.replace("&#174;","")
    content=content.replace("&#167;","")
    content=content.replace("&#169;","")
    content=content.replace("&#8220;","")
    content=content.replace("&#8221;","")
    content=content.replace("&rsquo;","")
    content=content.replace("&lsquo;","")
    content=content.replace("&sbquo;","")
    content=content.replace("&bdquo;","")
    content=content.replace("&ldquo;","")
    content=content.replace("&rdquo;","")
    content=content.replace("\'","")
    p = re.compile(r'&#\d{1,5};')
    content=p.sub("",content)
    p = re.compile(r'&#.{1,5};')
    content=p.sub("",content)
    content=content.replace("_"," ")
    content=content.replace("and/or","and or")
    content=content.replace("-\n"," ")
    p = re.compile(r'\s*-\s*')
    content=p.sub(" ",content)
    p = re.compile(r'(-|=)\s*')
    content=p.sub(" ",content)
    p = re.compile(r'\s\s*')
    content=p.sub(" ",content)
    p = re.compile(r'(\n\s*){3,}')
    content=p.sub("\n\n",content)
    p = re.compile(r'<.*?>')
    content=p.sub("",content)
    content = content.replace("'", "")
    return content

def cleancontent(content):
    content = headerclean(content)
    content = asciiclean(content)
    # content = tagsclean(content)# 很久，可能要改
    content = asciireplace("<xbrl", "</xbrl.*>", content)
    # content = table_clean('<table','</table>',content)
    content = cleannewline(content)
    content = cleanothers(content)
    return content

def findmda(content, path):
    item7={}
    item7[1]="item 7. managements discussion and analysis"
    item7[2]="item 7.managements discussion and analysis"
    item7[3]="item7. managements discussion and analysis"
    item7[4]="item7.managements discussion and analysis"
    item7[5]="item 7. management discussion and analysis"
    item7[6]="item 7.management discussion and analysis"
    item7[7]="item7. management discussion and analysis"
    item7[8]="item7.management discussion and analysis"
    item7[9]="item 7 managements discussion and analysis"
    item7[10]="item 7managements discussion and analysis"
    item7[11]="item7 managements discussion and analysis"
    item7[12]="item7managements discussion and analysis"
    item7[13]="item 7 management discussion and analysis"
    item7[14]="item 7management discussion and analysis"
    item7[15]="item7 management discussion and analysis"
    item7[16]="item7management discussion and analysis"
    item7[17]="item 7: managements discussion and analysis"
    item7[18]="item 7:managements discussion and analysis"
    item7[19]="item7: managements discussion and analysis"
    item7[20]="item7:managements discussion and analysis"
    item7[21]="item 7: management discussion and analysis"
    item7[22]="item 7:management discussion and analysis"
    item7[23]="item7: management discussion and analysis"
    item7[24]="item7:management discussion and analysis"
    item8={}
    item8[1]="item 8. financial statements"
    item8[2]="item 8.financial statements"
    item8[3]="item8. financial statements"
    item8[4]="item8.financial statements"
    item8[5]="item 8 financial statements"
    item8[6]="item 8financial statements"
    item8[7]="item8 financial statements"
    item8[8]="item8financial statements"
    item8[9]="item 8a. financial statements"
    item8[10]="item 8a.financial statements"
    item8[11]="item8a. financial statements"
    item8[12]="item8a.financial statements"
    item8[13]="item 8a financial statements"
    item8[14]="item 8afinancial statements"
    item8[15]="item8a financial statements"
    item8[16]="item8afinancial statements"
    item8[17]="item 8. consolidated financial statements"
    item8[18]="item 8.consolidated financial statements"
    item8[19]="item8. consolidated financial statements"
    item8[20]="item8.consolidated financial statements"
    item8[21]="item 8 consolidated  financial statements"
    item8[22]="item 8consolidated financial statements"
    item8[23]="item8 consolidated  financial statements"
    item8[24]="item8consolidated financial statements"
    item8[25]="item 8a. consolidated financial statements"
    item8[26]="item 8a.consolidated financial statements"
    item8[27]="item8a. consolidated financial statements"
    item8[28]="item8a.consolidated financial statements"
    item8[29]="item 8a consolidated financial statements"
    item8[30]="item 8aconsolidated financial statements"
    item8[31]="item8a consolidated financial statements"
    item8[32]="item8aconsolidated financial statements"
    item8[33]="item 8. audited financial statements"
    item8[34]="item 8.audited financial statements"
    item8[35]="item8. audited financial statements"
    item8[36]="item8.audited financial statements"
    item8[37]="item 8 audited financial statements"
    item8[38]="item 8audited financial statements"
    item8[39]="item8 audited financial statements"
    item8[40]="item8audited financial statements"
    item8[41]="item 8: financial statements"
    item8[42]="item 8:financial statements"
    item8[43]="item8: financial statements"
    item8[44]="item8:financial statements"
    item8[45]="item 8: consolidated financial statements"
    item8[46]="item 8:consolidated financial statements"
    item8[47]="item8: consolidated financial statements"
    item8[48]="item8:consolidated financial statements"
    
    look={" see ", " refer to ", " included in "," contained in "}
    a={}
    c={}
    
    lstr1=content.lower()
    for j in range(1,25):
        a[j]=[]
        for m in re.finditer(item7[j], lstr1):
            if not m:
                break
            else:
                substr1=lstr1[m.start()-20:m.start()]
                if not any(s in substr1 for s in look):   
                    #print substr1
                    b=m.start()
                    a[j].append(b)
    #print i
    
    list1=[]
    for value in a.values():
        for thing1 in value:
            list1.append(thing1)
    list1.sort()
    list1.append(len(lstr1))
    #print list1
           
    for j in range(1,49):
        c[j]=[]
        for m in re.finditer(item8[j], lstr1):
            if not m:
                break
            else:
                substr1=lstr1[m.start()-20:m.start()]
                if not any(s in substr1 for s in look):   
                    #print substr1
                    b=m.start()
                    c[j].append(b)
    list2=[]
    for value in c.values():
        for thing2 in value:
            list2.append(thing2)
    list2.sort()
    
    locations={}
    if list2==[]:
        print(path + " NO MD&A")
    else:
        if list1==[]:
            print(path + " NO MD&A")
        else:
            for k0 in range(len(list1)):
                locations[k0]=[]
                locations[k0].append(list1[k0])
            for k0 in range(len(locations)):
                for item in range(len(list2)):
                    if locations[k0][0]<=list2[item]:
                        locations[k0].append(list2[item])
                        break
                if len(locations[k0])==1:
                    del locations[k0]
    output = ''
    if locations=={}:
        with open(LOG,'a') as f:
            f.write(str(path)+"\t"+"0"+"\t"+"0\n")
            f.close()
    else:
        sections=0
        short = 0
        for k0 in range(len(locations)): 
            substring2=content[locations[k0][0]:locations[k0][1]]
            n_words=substring2.split()
            if len(n_words)>50:
                sections=sections+1
                output = output + substring2 + "\n"
            else:
                short = short+1
        with open(LOG,'a') as f:
                f.write(str(path)+"\t"+str(sections)+"\t"+str(short)+"\n")
                f.close()
    return output

def extract_item(path):
    identity = parse_identity(path)
    with open(path, 'r', encoding='utf-8', errors="replace") as f:
        content = f.read() 
    content = cleancontent(content)
    identity['mda'] = findmda(content, path)
    return identity

In [158]:
# Test speed

tic = time.time()
path = 'D:/Thesis_data/sec_edgar_filings/'
# path = 'D:/Thesis_data/Test2/'
company_path_list = [os.path.join(path, company_path) for company_path in os.listdir(path)] 
n_company = len(company_path_list)
n = 0
print('Amount of company: {}'.format(n_company))
for company_path in company_path_list:
    if (n % 10 == 0):
        print('n = {}, {:.2f}%, Time used: {:.2f} sec.'.format(n, n/n_company*100, time.time()-tic))
    if os.path.isdir(company_path + '/10-K/'):
        filename_list = [company_path+'/10-K/'+filename for filename in os.listdir(company_path + '/10-K/')]
        result = []
        result = [extract_item(filename) for filename in filename_list]
        with open(company_path+'/processed_final.pickle', 'wb') as f:
            pickle.dump(result, f)
    n = n + 1
print('Done.\n', 'Total time used: {:.2f} sec.'.format(time.time()-tic))

Amount of company: 5549
n = 0, 0.00%, Time used: 0.03 sec.
D:/Thesis_data/sec_edgar_filings/1000209/10-K/0001193125-10-055581.txt NO MD&A
n = 10, 0.18%, Time used: 54.19 sec.
D:/Thesis_data/sec_edgar_filings/1001316/10-K/0001144204-08-018580.txt NO MD&A
n = 20, 0.36%, Time used: 124.09 sec.
n = 30, 0.54%, Time used: 179.95 sec.
D:/Thesis_data/sec_edgar_filings/1002910/10-K/0000950137-07-003131.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1002910/10-K/0000950137-08-003079.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1002910/10-K/0001002910-05-000168.txt NO MD&A
n = 40, 0.72%, Time used: 283.50 sec.
n = 50, 0.90%, Time used: 432.94 sec.
n = 60, 1.08%, Time used: 586.03 sec.
n = 70, 1.26%, Time used: 669.15 sec.
D:/Thesis_data/sec_edgar_filings/1007273/10-K/0001144204-10-012003.txt NO MD&A
n = 80, 1.44%, Time used: 748.88 sec.
n = 90, 1.62%, Time used: 868.66 sec.
n = 100, 1.80%, Time used: 987.37 sec.
n = 110, 1.98%, Time used: 1134.04 sec.
n = 120, 2.16%, Time used: 1273.33 sec.
D:/Thes

n = 830, 14.96%, Time used: 8135.03 sec.
n = 840, 15.14%, Time used: 8269.11 sec.
n = 850, 15.32%, Time used: 8364.93 sec.
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001096343-13-000005.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001096343-14-000006.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001096343-15-000013.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001096343-16-000185.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001096343-17-000046.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001096343-18-000042.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001096343-19-000074.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001193125-05-039831.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001193125-06-045092.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001193125-07-043689.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1096343/10-K/0001193125-08-042863.txt NO MD&A
D:/Thesi

n = 1680, 30.28%, Time used: 15254.79 sec.
n = 1690, 30.46%, Time used: 15308.77 sec.
n = 1700, 30.64%, Time used: 15354.82 sec.
D:/Thesis_data/sec_edgar_filings/1296205/10-K/0001296205-19-000020.txt NO MD&A
n = 1710, 30.82%, Time used: 15440.27 sec.
n = 1720, 31.00%, Time used: 15537.09 sec.
n = 1730, 31.18%, Time used: 15617.28 sec.
D:/Thesis_data/sec_edgar_filings/1299139/10-K/0001140361-10-013848.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1299704/10-K/0001047469-10-001876.txt NO MD&A
n = 1740, 31.36%, Time used: 15687.78 sec.
D:/Thesis_data/sec_edgar_filings/1301206/10-K/0001144204-09-020883.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1301206/10-K/0001144204-10-020265.txt NO MD&A
n = 1750, 31.54%, Time used: 15758.30 sec.
D:/Thesis_data/sec_edgar_filings/1302028/10-K/0001193125-05-106393.txt NO MD&A
n = 1760, 31.72%, Time used: 15840.57 sec.
D:/Thesis_data/sec_edgar_filings/1303330/10-K/0001079973-08-000316.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/1303330/10-K/0001144204-09-

n = 2950, 53.16%, Time used: 24308.07 sec.
n = 2960, 53.34%, Time used: 24364.90 sec.
n = 2970, 53.52%, Time used: 24419.35 sec.
D:/Thesis_data/sec_edgar_filings/1518832/10-K/0001518832-19-000005.txt NO MD&A
n = 2980, 53.70%, Time used: 24499.58 sec.
n = 2990, 53.88%, Time used: 24579.74 sec.
n = 3000, 54.06%, Time used: 24637.82 sec.
n = 3010, 54.24%, Time used: 24717.06 sec.
D:/Thesis_data/sec_edgar_filings/1526520/10-K/0001193125-12-117448.txt NO MD&A
n = 3020, 54.42%, Time used: 24772.31 sec.
n = 3030, 54.60%, Time used: 24852.92 sec.
n = 3040, 54.78%, Time used: 24909.58 sec.
n = 3050, 54.96%, Time used: 24998.95 sec.
D:/Thesis_data/sec_edgar_filings/1534675/10-K/0001144204-12-034903.txt NO MD&A
n = 3060, 55.15%, Time used: 25069.61 sec.
n = 3070, 55.33%, Time used: 25174.05 sec.
n = 3080, 55.51%, Time used: 25228.75 sec.
n = 3090, 55.69%, Time used: 25280.86 sec.
n = 3100, 55.87%, Time used: 25321.21 sec.
n = 3110, 56.05%, Time used: 25372.36 sec.
n = 3120, 56.23%, Time used: 254

D:/Thesis_data/sec_edgar_filings/730464/10-K/0001144204-14-052733.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/730464/10-K/0001144204-15-052225.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/730464/10-K/0001144204-16-121173.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/730464/10-K/0001144204-17-044976.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/730464/10-K/0001144204-18-046216.txt NO MD&A
n = 3990, 71.90%, Time used: 36664.16 sec.
D:/Thesis_data/sec_edgar_filings/731012/10-K/0000731012-19-000040.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/731653/10-K/0001144204-10-015171.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/731939/10-K/0000950123-10-015600.txt NO MD&A
n = 4000, 72.09%, Time used: 36833.00 sec.
D:/Thesis_data/sec_edgar_filings/733269/10-K/0000733269-19-000022.txt NO MD&A
n = 4010, 72.27%, Time used: 37010.45 sec.
n = 4020, 72.45%, Time used: 37275.31 sec.
D:/Thesis_data/sec_edgar_filings/740664/10-K/0001144204-12-004278.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/740664/10-K/000

D:/Thesis_data/sec_edgar_filings/831001/10-K/0001104659-14-015152.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/831001/10-K/0001193125-07-038505.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/831001/10-K/0001193125-08-036445.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/831001/10-K/0001193125-09-041237.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/831001/10-K/0001206774-10-000406.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/831001/10-K/0001206774-11-000316.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/831001/10-K/0001206774-12-000799.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/831001/10-K/0001206774-13-000852.txt NO MD&A
n = 4500, 81.10%, Time used: 43464.26 sec.
n = 4510, 81.28%, Time used: 43579.73 sec.
D:/Thesis_data/sec_edgar_filings/833320/10-K/0000950129-05-001767.txt NO MD&A
D:/Thesis_data/sec_edgar_filings/833320/10-K/0000950129-06-002048.txt NO MD&A
n = 4520, 81.46%, Time used: 43723.81 sec.
n = 4530, 81.64%, Time used: 43798.17 sec.
n = 4540, 81.82%, Time used: 43873.90 sec.
n = 4

n = 5400, 97.31%, Time used: 53421.73 sec.
n = 5410, 97.50%, Time used: 53524.02 sec.
n = 5420, 97.68%, Time used: 53618.17 sec.
D:/Thesis_data/sec_edgar_filings/933590/10-K/0001144204-11-018595.txt NO MD&A
n = 5430, 97.86%, Time used: 53736.81 sec.
n = 5440, 98.04%, Time used: 53833.25 sec.
n = 5450, 98.22%, Time used: 53942.36 sec.
n = 5460, 98.40%, Time used: 54122.16 sec.
D:/Thesis_data/sec_edgar_filings/940332/10-K/0000940332-05-000038.txt NO MD&A
n = 5470, 98.58%, Time used: 54225.15 sec.
n = 5480, 98.76%, Time used: 54318.01 sec.
D:/Thesis_data/sec_edgar_filings/944695/10-K/0001193125-12-083502.txt NO MD&A
n = 5490, 98.94%, Time used: 54452.54 sec.
n = 5500, 99.12%, Time used: 54541.66 sec.
n = 5510, 99.30%, Time used: 54643.10 sec.
n = 5520, 99.48%, Time used: 54753.83 sec.
n = 5530, 99.66%, Time used: 54856.90 sec.
n = 5540, 99.84%, Time used: 55006.17 sec.
Done.
 Total time used: 55085.02 sec.


* cleanhead:23.3
* cleanascii:81.9
* asciireplace:3.1
* table_clean:14.8
* cleannewline:2.6
* cleanothers:33.3

In [124]:
i = 0
# test_nomda_path = log.loc[i,'path']
test_nomda_path = 'D:/Thesis_data/Test2/66479/10-K/0001193125-07-043177.txt '
def test_extract_item(path):
    content=''
    identity = parse_identity(path)
    with open(path, 'r', encoding='utf-8', errors="replace") as f:
        content = f.read() 
    content = cleancontent(content)
    identity['mda'] = findmda(content, path)
    return identity, content
item, content = test_extract_item(test_nomda_path)

In [128]:
len(item['mda'].split())

13268

In [126]:
re.findall(r'item.{50}',content, flags=re.IGNORECASE)

['Item 405 of Regulation S K is not contained herein and',
 'Item 1. Business 3 Item 1A. Risk Factors 21 Item 1B. U',
 'Item 2. Properties 31 Item 3. Legal Proceedings 32 Ite',
 'Item 5. Market for Registrants Common Stock, Related S',
 'Item 6. Selected Financial Data 33 Item 7. Managements',
 'Item 7A. Quantitative and Qualitative Disclosures Abou',
 'Item 8. Financial Statements and Supplementary Data 57',
 'Item 9. Changes In and Disagreements With Accountants ',
 'Item 9A. Controls and Procedures 102 Item 9B. Other In',
 'Item 10. Directors, Executive Officers and Corporate G',
 'Item 11. Executive Compensation 103 Item 12. Security ',
 'Item 13. Certain Relationships and Related Transaction',
 'Item 14. Principal Accountant Fees and Services 104 PA',
 'Item 15. Exhibits and Financial Statement Schedules 10',
 'Item 1. Business. Overview Millipore is a global life ',
 'Item 2 of this Form 10 K. Our global supply chain init',
 'Item 1A of this Form 10 K. Raw Materials Our products 

In [136]:
log = pd.read_csv('D:/Thesis_data/log.txt',sep='\t',names=['path','No','short'])

In [137]:
sum(log['No'] == 0)

72

In [131]:
log

,path,No,short
0,D:/Thesis_data/Test2/64670/10-K/0000064670-13-...,1,1
1,D:/Thesis_data/Test2/64670/10-K/0000064670-14-...,1,1
2,D:/Thesis_data/Test2/64670/10-K/0000897101-05-...,1,0
3,D:/Thesis_data/Test2/64670/10-K/0000897101-06-...,1,0
4,D:/Thesis_data/Test2/64670/10-K/0000897101-07-...,1,0
5,D:/Thesis_data/Test2/64670/10-K/0000897101-08-...,1,0
6,D:/Thesis_data/Test2/64670/10-K/0000897101-09-...,1,0
7,D:/Thesis_data/Test2/64670/10-K/0000897101-10-...,1,0
8,D:/Thesis_data/Test2/64670/10-K/0000897101-11-...,1,0
9,D:/Thesis_data/Test2/64670/10-K/0000897101-12-...,1,1


In [144]:
df = pd.DataFrame(pd.read_pickle('D:/Thesis_data/Test2/64670/processed.pickle'))

In [152]:
df['mda'][1]

'Item 7. Managements Discussion and Analysis of Financial Condition and Results of OperationsUnderstanding Our Financial InformationThe following discussion and analysis provides information management believes to be relevant to understanding the financial condition and results of operations of Medtronic, Inc. and its subsidiaries (Medtronic or the Company, or we, us, or our). You should read this discussion and analysis along with our consolidated financial statements and related notes thereto as of April 25, 2014 and April 26, 2013 and for each of the three fiscal years ended April 25, 2014, April 26, 2013, and April 27, 2012.Beginning in the third quarter of fiscal year 2012, the results of operations, assets, and liabilities of the Physio Control business, which were previously presented as a component of the Cardiac and Vascular Group operating segment, are classified as discontinued operations. All information in the following managements discussion and analysis of financial cond